**Domain Based Inteligent Network Intrusion Detection System**

In [ ]:
os.chdir('/content/drive/MyDrive/domain-knowledge')
from sklearn.utils import shuffle
start = time.time()
# import data
dataset = pd.read_csv('data/combined_sampled7.csv', sep=',', dtype='unicode')

Import Data

In [ ]:
import os
import cv2
import pandas as pd
import time
import pandas as pd 
import numpy as np
import scipy
from scipy.stats import pearsonr 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset.head()

,id,dur,xProt,xServ,xState,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,Class_all,Class
0,1,0.000011,2,999,1,2,0,496,0,90909.0902,254,0,180363632,0,0,0,0.011,0,0,0,0,0,0,0,0,0,0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,0,0
1,2,0.000008,2,999,1,2,0,1762,0,125000.0003,254,0,881000000,0,0,0,0.008,0,0,0,0,0,0,0,0,0,0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,0,0
2,3,0.000005,2,999,1,2,0,1068,0,200000.0051,254,0,854400000,0,0,0,0.005,0,0,0,0,0,0,0,0,0,0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,0,0
3,4,0.000006,2,999,1,2,0,900,0,166666.6608,254,0,600000000,0,0,0,0.006,0,0,0,0,0,0,0,0,0,0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,0,0
4,5,0.00001,2,999,1,2,0,2126,0,100000.0025,254,0,850400000,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,0,0


In [ ]:
dataset = dataset.iloc[:, 1:]  
max_value = 655453030.0
X = dataset.iloc[:,0:-1].values
X_columns = dataset.iloc[:,0:-1].columns.values
y = dataset.iloc[:,-1].values

from sklearn.preprocessing import LabelEncoder
df_dump_part1 = pd.DataFrame(X, columns=X_columns)
df_dump_part2 = pd.DataFrame(y, columns=['Class'])   
df_dump = pd.concat([df_dump_part1,df_dump_part2], axis = 1)     
df_dump.to_csv("data/data_preprocessed_numerical.csv",encoding='utf-8', index = False)   
end = time.time()
print("checkpoint 1:", end-start)


checkpoint 1: 73.97590041160583


Preprocessing 

In [ ]:
start = time.time()
# Label Encoding
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

X = np.array(X, dtype=float) 
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        k = X[i,j]
        if not np.isfinite(k):
            X[i,j] = max_value
        if np.isnan(k):
            X[i,j] = 0.0

# Standard Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_ = sc.fit_transform(X[:, 1:-1]) 
X = np.hstack((X[:,[0,-1]],X_))

del X_

index = np.arange(len(X)).reshape(len(X),1)
X = np.hstack((index,X))

# Train Test Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42,stratify=y)


col_l = ['index','index_old', 'Class_all']
for i in range(1,len(X_columns)-1):  
    col_l.append(X_columns[i])



df_dump_part1 = pd.DataFrame(X_train, columns=col_l)
df_dump_part2 = pd.DataFrame(y_train, columns=['Class'])   
df_dump = pd.concat([df_dump_part1,df_dump_part2], axis = 1)     
df_dump.to_csv("data/data_preprocessed_numerical_train_all_features.csv",encoding='utf-8')  

df_dump_part1 = pd.DataFrame(X_test, columns=col_l)
df_dump_part2 = pd.DataFrame(y_test, columns=['Class'])   
df_dump = pd.concat([df_dump_part1,df_dump_part2], axis = 1)     
df_dump.to_csv("data/data_preprocessed_numerical_test_all_features.csv",encoding='utf-8')  

del df_dump_part1
del df_dump_part2
del df_dump

end = time.time()
print("checkpoint 2:", end-start)

checkpoint 2: 19.201738834381104


Unknown attack data trainning

In [ ]:


#import modules
import pandas as pd   
import numpy as np
import time
start = time.time()


config_file = 'config.txt'
config = pd.read_csv(config_file,sep=',', index_col =None)
attack_id = config.iloc[2,1]
print("attack_id: ", attack_id)

# import data
dataset_train = pd.read_csv('data/data_preprocessed_numerical_train_all_features.csv', sep=',')
dataset_test = pd.read_csv('data/data_preprocessed_numerical_test_all_features.csv', sep=',')

# delete all records from training set containing the attack in attack_id
dataset_train = dataset_train[dataset_train['Class_all'] != attack_id]


feature_selected = ['xProt','xServ','xState','rate','sttl','sload','sinpkt','dload','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','dmean','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm',
'ct_src_ltm','ct_srv_dst','Class']

dataset_train = dataset_train[feature_selected]
dataset_test = dataset_test[feature_selected]
from sklearn import preprocessing

dataset_train_class = np.array(dataset_train['Class'])
xx = dataset_train.iloc[:,0:-1].values
xx_columns = dataset_train.columns
xx_columns = xx_columns[0:-1]
min_max_scaler = preprocessing.MinMaxScaler()
xx_scaled = min_max_scaler.fit_transform(xx)
dataset_train = pd.DataFrame(xx_scaled)
df_part1 = pd.DataFrame(xx_scaled, columns=xx_columns)
df_part2 = pd.DataFrame(dataset_train_class, columns=['Class'])   
dataset_train = pd.concat([df_part1,df_part2], axis = 1) 
 
# Creating new CIA features

dataset_train.insert(loc = 25, column = 'C', value = np.zeros(len(dataset_train)))
dataset_train.insert(loc = 26, column = 'I', value = np.zeros(len(dataset_train)))
dataset_train.insert(loc = 27, column = 'A', value = np.zeros(len(dataset_train)))

dataset_train['C'] = dataset_train['C'] + dataset_train['xProt']*.25
dataset_train['I'] = dataset_train['I'] + dataset_train['xProt']*.25
dataset_train['A'] = dataset_train['A'] - dataset_train['xProt']*.25

dataset_train['C'] = dataset_train['C'] - dataset_train['xServ']*.3
dataset_train['A'] = dataset_train['A'] - dataset_train['xServ']*.3

dataset_train['I'] = dataset_train['I'] - dataset_train['xState']*.4
dataset_train['C'] = dataset_train['C'] - dataset_train['xState']*.4

dataset_train['C'] = dataset_train['C'] + dataset_train['rate']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['rate']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['rate']*.3

dataset_train['C'] = dataset_train['C'] + dataset_train['sttl']*.5
dataset_train['I'] = dataset_train['I'] + dataset_train['sttl']*.5
dataset_train['A'] = dataset_train['A'] + dataset_train['sttl']*.5

dataset_train['C'] = dataset_train['C'] + dataset_train['sload']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['sload']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['sload']*.3


dataset_train['C'] = dataset_train['C'] - dataset_train['dload']*.3
dataset_train['I'] = dataset_train['I'] - dataset_train['dload']*.3
dataset_train['A'] = dataset_train['A'] - dataset_train['dload']*.3

dataset_train['A'] = dataset_train['A'] - dataset_train['sinpkt']*.1
dataset_train['C'] = dataset_train['C'] - dataset_train['sinpkt']*.1

dataset_train['A'] = dataset_train['A'] - dataset_train['swin']*.4 
dataset_train['C'] = dataset_train['C'] - dataset_train['swin']*.4

dataset_train['C'] = dataset_train['C'] - dataset_train['stcpb']*.3
dataset_train['I'] = dataset_train['I'] - dataset_train['stcpb']*.3
dataset_train['A'] = dataset_train['A'] - dataset_train['stcpb']*.3

dataset_train['I'] = dataset_train['I'] - dataset_train['dtcpb']*.3
dataset_train['A'] = dataset_train['A'] - dataset_train['dtcpb']*.3

dataset_train['A'] = dataset_train['A'] - dataset_train['dwin']*.3
dataset_train['C'] = dataset_train['C'] - dataset_train['dwin']*.3

dataset_train['C'] = dataset_train['C'] - dataset_train['tcprtt']*.1
dataset_train['I'] = dataset_train['I'] - dataset_train['tcprtt']*.1
dataset_train['A'] = dataset_train['A'] - dataset_train['tcprtt']*.1

dataset_train['C'] = dataset_train['C'] - dataset_train['synack']*.1
dataset_train['I'] = dataset_train['I'] - dataset_train['synack']*.1
dataset_train['A'] = dataset_train['A'] - dataset_train['synack']*.1

dataset_train['C'] = dataset_train['C'] - dataset_train['ackdat']*.1
dataset_train['I'] = dataset_train['I'] - dataset_train['ackdat']*.1
dataset_train['A'] = dataset_train['A'] - dataset_train['ackdat']*.1

dataset_train['C'] = dataset_train['C'] - dataset_train['dmean']*.2
dataset_train['I'] = dataset_train['I'] - dataset_train['dmean']*.2

dataset_train['C'] = dataset_train['C'] + dataset_train['ct_srv_src']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_srv_src']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['ct_srv_src']*.3

dataset_train['C'] = dataset_train['C'] + dataset_train['ct_state_ttl']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['ct_state_ttl']*.3

dataset_train['A'] = dataset_train['A'] + dataset_train['ct_dst_ltm']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_dst_ltm']*.3

dataset_train['C'] = dataset_train['C'] + dataset_train['ct_src_dport_ltm']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_src_dport_ltm']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['ct_src_dport_ltm']*.3

dataset_train['C'] = dataset_train['C'] + dataset_train['ct_dst_sport_ltm']*.4
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_dst_sport_ltm']*.4

dataset_train['C'] = dataset_train['C'] + dataset_train['ct_dst_src_ltm']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_dst_src_ltm']*.3


dataset_train['C'] = dataset_train['C'] + dataset_train['ct_src_ltm']*.3
dataset_train['I'] = dataset_train['I'] + dataset_train['ct_src_ltm']*.3
dataset_train['A'] = dataset_train['A'] + dataset_train['ct_src_ltm']*.3

dataset_train['A'] = dataset_train['A'] + dataset_train['ct_srv_dst']*.3



dataset_test_class = np.array(dataset_test['Class'])
xx = dataset_test.iloc[:,0:-1].values
xx_columns = dataset_test.columns
xx_columns = xx_columns[0:-1]
min_max_scaler = preprocessing.MinMaxScaler()
xx_scaled = min_max_scaler.fit_transform(xx)
dataset_test = pd.DataFrame(xx_scaled)
df_part1 = pd.DataFrame(xx_scaled, columns=xx_columns)
df_part2 = pd.DataFrame(dataset_test_class, columns=['Class'])   
dataset_test = pd.concat([df_part1,df_part2], axis = 1) 


dataset_test.insert(loc = 25, column = 'C', value = np.zeros(len(dataset_test)))
dataset_test.insert(loc = 26, column = 'I', value = np.zeros(len(dataset_test)))
dataset_test.insert(loc = 27, column = 'A', value = np.zeros(len(dataset_test)))

dataset_test['C'] = dataset_test['C'] + dataset_test['xProt']*.25
dataset_test['I'] = dataset_test['I'] + dataset_test['xProt']*.25
dataset_test['A'] = dataset_test['A'] - dataset_test['xProt']*.25

dataset_test['C'] = dataset_test['C'] - dataset_test['xServ']*.3
dataset_test['A'] = dataset_test['A'] - dataset_test['xServ']*.3

dataset_test['I'] = dataset_test['I'] - dataset_test['xState']*.4
dataset_test['C'] = dataset_test['C'] - dataset_test['xState']*.4

dataset_test['C'] = dataset_test['C'] + dataset_test['rate']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['rate']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['rate']*.3

dataset_test['C'] = dataset_test['C'] + dataset_test['sttl']*.5
dataset_test['I'] = dataset_test['I'] + dataset_test['sttl']*.5
dataset_test['A'] = dataset_test['A'] + dataset_test['sttl']*.5

dataset_test['C'] = dataset_test['C'] + dataset_test['sload']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['sload']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['sload']*.3

dataset_test['C'] = dataset_test['C'] - dataset_test['dload']*.3
dataset_test['I'] = dataset_test['I'] - dataset_test['dload']*.3
dataset_test['A'] = dataset_test['A'] - dataset_test['dload']*.3

dataset_test['A'] = dataset_test['A'] - dataset_test['sinpkt']*.1
dataset_test['C'] = dataset_test['C'] - dataset_test['sinpkt']*.1

dataset_test['A'] = dataset_test['A'] - dataset_test['swin']*.4
dataset_test['C'] = dataset_test['C'] - dataset_test['swin']*.4

dataset_test['C'] = dataset_test['C'] - dataset_test['stcpb']*.3
dataset_test['I'] = dataset_test['I'] - dataset_test['stcpb']*.3
dataset_test['A'] = dataset_test['A'] - dataset_test['stcpb']*.3

dataset_test['I'] = dataset_test['I'] - dataset_test['dtcpb']*.3
dataset_test['A'] = dataset_test['A'] - dataset_test['dtcpb']*.3

dataset_test['A'] = dataset_test['A'] - dataset_test['dwin']*.3
dataset_test['C'] = dataset_test['C'] - dataset_test['dwin']*.3

dataset_test['C'] = dataset_test['C'] - dataset_test['tcprtt']*.1
dataset_test['I'] = dataset_test['I'] - dataset_test['tcprtt']*.1
dataset_test['A'] = dataset_test['A'] - dataset_test['tcprtt']*.1

dataset_test['C'] = dataset_test['C'] - dataset_test['synack']*.1
dataset_test['I'] = dataset_test['I'] - dataset_test['synack']*.1
dataset_test['A'] = dataset_test['A'] - dataset_test['synack']*.1

dataset_test['C'] = dataset_test['C'] - dataset_test['ackdat']*.1
dataset_test['I'] = dataset_test['I'] - dataset_test['ackdat']*.1
dataset_test['A'] = dataset_test['A'] - dataset_test['ackdat']*.1

dataset_test['C'] = dataset_test['C'] - dataset_test['dmean']*.2
dataset_test['I'] = dataset_test['I'] - dataset_test['dmean']*.2

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_srv_src']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_srv_src']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['ct_srv_src']*.3

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_state_ttl']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['ct_state_ttl']*.3

dataset_test['A'] = dataset_test['A'] + dataset_test['ct_dst_ltm']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_dst_ltm']*.3

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_src_dport_ltm']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_src_dport_ltm']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['ct_src_dport_ltm']*.3

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_dst_sport_ltm']*.4
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_dst_sport_ltm']*.4

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_dst_src_ltm']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_dst_src_ltm']*.3

dataset_test['C'] = dataset_test['C'] + dataset_test['ct_src_ltm']*.3
dataset_test['I'] = dataset_test['I'] + dataset_test['ct_src_ltm']*.3
dataset_test['A'] = dataset_test['A'] + dataset_test['ct_src_ltm']*.3

dataset_test['A'] = dataset_test['A'] + dataset_test['ct_srv_dst']*.3

feature_selected = ['C','I','A','Class']
dataset_train = dataset_train[feature_selected]
dataset_test = dataset_test[feature_selected]

#dataset_train = dataset_train.drop(['Unnamed: 0', 'index', 'index_old', 'Class_all'], axis=1)
#dataset_test = dataset_test.drop(['Unnamed: 0', 'index', 'index_old', 'Class_all'], axis=1)

X_train = dataset_train.iloc[:,0:-1].values
y_train = dataset_train.iloc[:,-1].values

X_test = dataset_test.iloc[:,0:-1].values
y_test = dataset_test.iloc[:,-1].values

end = time.time()
print("checkpoint 1:", end-start)


# save the fully processed data as binary for future use in any ML algorithm without any more preprocessing. 
np.save('data/data_fully_processed_X_train_domain_features.npy',X_train)
np.save('data/data_fully_processed_y_train_domain_features.npy',y_train)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

# save the fully processed data as binary for future use in any ML algorithm without any more preprocessing. 
np.save('data/data_fully_processed_X_test_domain_features.npy',X_test)
np.save('data/data_fully_processed_y_test_domain_features.npy',y_test)


end = time.time()
print("checkpoint 2:", end-start)

################oversampling the minority class of training set #########

from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

# save the fully processed data as binary for future use in any ML algorithm without any more preprocessing. 
np.save('data/data_fully_processed_X_train_resampled_domain_features.npy',X_train_res)
np.save('data/data_fully_processed_y_train_resampled_domain_features.npy',y_train_res)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


attack_id:  8
checkpoint 1: 1.3323984146118164
Before OverSampling, counts of label '1': 31470
Before OverSampling, counts of label '0': 25900 

checkpoint 2: 1.594419002532959


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


After OverSampling, the shape of train_X: (62940, 3)
After OverSampling, the shape of train_y: (62940,) 

After OverSampling, counts of label '1': 31470
After OverSampling, counts of label '0': 31470


In [ ]:
dataset_train.head()

,C,I,A,Class
0,0.424268,0.634613,0.413150,0
1,0.579145,0.789489,0.589886,0
2,1.776649,1.859188,1.809109,1
3,-0.780204,-0.024357,-0.909212,0
4,1.419030,1.444672,1.276252,1


In [ ]:
dataset_test.head()

,C,I,A,Class
0,0.366125,0.577890,0.383396,0
1,0.795144,1.071615,0.666124,1
2,-0.682800,0.057375,-0.814922,1
3,-0.298076,0.001923,-0.301922,0
4,0.961122,0.915568,0.986069,1


Classification Models

In [ ]:
# Neural Network

import pandas as pd   
import numpy as np
import time

config_file = 'config.txt'
config = pd.read_csv(config_file,sep=',', index_col =None)
resample_data = config.iloc[0,1] #0 or 1
feature_set = config.iloc[1,1] # 1 = full features, 2 = selected, 3 = domain
attack_id = config.iloc[2,1]

del config

print("Neural Network:",resample_data)
start = time.time()



f_X_train = 'data/data_fully_processed_X_train'
f_y_train = 'data/data_fully_processed_y_train'
f_X_test = 'data/data_fully_processed_X_test'
f_y_test = 'data/data_fully_processed_y_test'

if resample_data == 1:
    f_X_train = f_X_train + "_resampled"
    f_y_train = f_y_train + "_resampled"

if feature_set == 1:
    f_X_train = f_X_train + "_all_features"
    f_y_train = f_y_train + "_all_features"
    f_X_test = f_X_test + "_all_features"
    f_y_test = f_y_test + "_all_features"
elif feature_set == 2:
    f_X_train = f_X_train + "_selected_features"
    f_y_train = f_y_train + "_selected_features"
    f_X_test = f_X_test + "_selected_features"
    f_y_test = f_y_test + "_selected_features"
else:
    f_X_train = f_X_train + "_domain_features"
    f_y_train = f_y_train + "_domain_features"
    f_X_test = f_X_test + "_domain_features"
    f_y_test = f_y_test + "_domain_features"  

    

f_X_train = f_X_train + ".npy"
f_y_train = f_y_train + ".npy"
f_X_test = f_X_test + ".npy"
f_y_test = f_y_test + ".npy"


print(f_X_train)
print(f_y_train)
print(f_X_test)
print(f_y_test)

X_train = np.load(f_X_train)
y_train = np.load(f_y_train)
X_test = np.load(f_X_test)
y_test = np.load(f_y_test)


import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout



input_dim = X_train.shape[1]
units = int(input_dim/2)+1

classifier = Sequential()

classifier.add(Dense(units = units, kernel_initializer = 'uniform', activation = 'relu', input_dim = input_dim))
classifier.add(Dropout(rate = 0.1))

classifier.add(Dense(units = units, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 1000, epochs = 200) 
y_pred_probab = classifier.predict(X_test)

y_pred = np.zeros(len(y_test))
for i in range(len(y_test)):
    if y_pred_probab[i] > .5:
        y_pred[i] =1
        

df_y_pred = pd.DataFrame(y_pred, columns=['y_pred'])
df_y_pred.to_csv("data2/y_pred.csv",encoding='utf-8')   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, roc_curve, auc, precision_recall_curve 
from sklearn.metrics import precision_recall_fscore_support
cm = confusion_matrix(y_test, y_pred)

acsc = accuracy_score(y_test, y_pred) 
df_cm = pd.DataFrame([[cm[1][1], cm[0][0],cm[0][1], cm[1][0]]], 
                        index=[0],
                        columns=['True Positives','True Negatives', 'False Positives', 'False Negatives'])
print(df_cm)

precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred,average='binary')


fpr, tpr, thresholds = roc_curve(y_test, y_pred_probab, pos_label=1)
roc_auc = auc(fpr,tpr) 
prc_precision, prc_recall, prc_thresholds = precision_recall_curve(y_test, y_pred_probab)
prc_auc = ''
df_metrics = pd.DataFrame([[acsc, precision, recall, fscore,roc_auc]], 
                        index=[0],
                        columns=['accuracy','precision', 'recall', 'fscore', 'ROC-AUC'])

print(df_metrics)


end = time.time()
print(df_metrics.iloc[0][0],',',df_metrics.iloc[0][1],',',df_metrics.iloc[0][2],',',df_metrics.iloc[0][3],',',df_metrics.iloc[0][4],',',df_cm.iloc[0][0],',',df_cm.iloc[0][1],',',df_cm.iloc[0][2],',',df_cm.iloc[0][3],',', end-start)

print("Time taken:", end-start)

Neural Network: 1
data/data_fully_processed_X_train_resampled_domain_features.npy
data/data_fully_processed_y_train_resampled_domain_features.npy
data/data_fully_processed_X_test_domain_features.npy
data/data_fully_processed_y_test_domain_features.npy
Epoch 1/200
63/63 [==============================] - 13s 1ms/step - loss: 0.6930 - accuracy: 0.5173
Epoch 2/200
63/63 [==============================] - 0s 1ms/step - loss: 0.6896 - accuracy: 0.7057
Epoch 3/200
63/63 [==============================] - 0s 1ms/step - loss: 0.6725 - accuracy: 0.7091
Epoch 4/200
63/63 [==============================] - 0s 1ms/step - loss: 0.6382 - accuracy: 0.7093
Epoch 5/200
63/63 [==============================] - 0s 1ms/step - loss: 0.5974 - accuracy: 0.7117
Epoch 6/200
63/63 [==============================] - 0s 1ms/step - loss: 0.5652 - accuracy: 0.7173
Epoch 7/200
63/63 [==============================] - 0s 1ms/step - loss: 0.5526 - accuracy: 0.7140
Epoch 8/200
63/63 [==============================] - 0

In [ ]:
df_y_pred.head()
df_y_pred.to_csv("data2/y_pred.csv",encoding='utf-8')

In [ ]:
#Random Forest

print("RF:",resample_data)
start = time.time()

from sklearn.model_selection import KFold, cross_val_score

## random forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10,criterion="gini", n_jobs=4)



# Fitting classifier to the Training set    
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

#dump y_pred for future use ( to calculate percent of attack detected in case of experiment 2 where we exclude one attack from training set)
df_y_pred = pd.DataFrame(y_pred, columns=['y_pred'])
df_y_pred.to_csv("data/y_pred.csv",encoding='utf-8')   

# Performance metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, roc_curve, auc, precision_recall_curve 
from sklearn.metrics import precision_recall_fscore_support
cm = confusion_matrix(y_test, y_pred)

#accuracy -number of instance correctly classified
acsc = accuracy_score(y_test, y_pred) 
df_cm = pd.DataFrame([[cm[1][1], cm[0][0],cm[0][1], cm[1][0]]], 
                        index=[0],
                        columns=['True Positives','True Negatives', 'False Positives', 'False Negatives'])
print(df_cm)
#precision, recall, fscore, support
precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred,average='binary')

#balanced_as = balanced_accuracy_score(y_test, y_pred)

fpr, tpr, thresholds = roc_curve(y_test, classifier.predict_proba(X_test)[:,1], pos_label=1)
roc_auc = auc(fpr,tpr) # ROC-AUC

#precision recall AUC ->PRC
prc_precision, prc_recall, prc_thresholds = precision_recall_curve(y_test, classifier.predict_proba(X_test)[:,1])
#prc_auc = auc(prc_precision,prc_recall)
prc_auc = ''
df_metrics = pd.DataFrame([[acsc, precision, recall, fscore,roc_auc]], 
                        index=[0],
                        columns=['accuracy','precision', 'recall', 'fscore', 'ROC-AUC'])

print(df_metrics)
end = time.time()

print(df_metrics.iloc[0][0],',',df_metrics.iloc[0][1],',',df_metrics.iloc[0][2],',',df_metrics.iloc[0][3],',',df_metrics.iloc[0][4],',',df_cm.iloc[0][0],',',df_cm.iloc[0][1],',',df_cm.iloc[0][2],',',df_cm.iloc[0][3],',', end-start)

print("Time taken:", end-start)

RF: 1
   True Positives  True Negatives  False Positives  False Negatives
0           11796            8474             2626             1804
   accuracy  precision    recall   fscore   ROC-AUC
0  0.820648   0.817917  0.867353  0.84191  0.912861
0.8206477732793522 , 0.8179170711413118 , 0.8673529411764705 , 0.8419099279137819 , 0.9128605359035504 , 11796 , 8474 , 2626 , 1804 , 1.163201093673706
Time taken: 1.163201093673706


In [ ]:
# Logistic Regression
print("LR:",resample_data)
start = time.time()


from sklearn.model_selection import KFold, cross_val_score

## random forest
from sklearn.linear_model import LogisticRegression
#n_jobs = -1 means use all processors.
classifier = LogisticRegression(random_state=0, solver='saga',n_jobs=4)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
#dump y_pred for future use ( to calculate percent of attack detected in case of experiment 2 where we exclude one attack from training set)
df_y_pred = pd.DataFrame(y_pred, columns=['y_pred'])
df_y_pred.to_csv("data/y_pred.csv",encoding='utf-8')   



# Performance metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, roc_curve, auc, precision_recall_curve 
from sklearn.metrics import precision_recall_fscore_support
cm = confusion_matrix(y_test, y_pred)

#accuracy -number of instance correctly classified
acsc = accuracy_score(y_test, y_pred) 
df_cm = pd.DataFrame([[cm[1][1], cm[0][0],cm[0][1], cm[1][0]]], 
                        index=[0],
                        columns=['True Positives','True Negatives', 'False Positives', 'False Negatives'])
print(df_cm)
#precision, recall, fscore, support
precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred,average='binary')

#balanced_as = balanced_accuracy_score(y_test, y_pred)

fpr, tpr, thresholds = roc_curve(y_test, classifier.predict_proba(X_test)[:,1], pos_label=1)
roc_auc = auc(fpr,tpr) # ROC-AUC

#precision recall AUC ->PRC
prc_precision, prc_recall, prc_thresholds = precision_recall_curve(y_test, classifier.predict_proba(X_test)[:,1])
#prc_auc = auc(prc_precision,prc_recall)
prc_auc=''
df_metrics = pd.DataFrame([[acsc, precision, recall, fscore,roc_auc]], 
                        index=[0],
                        columns=['accuracy','precision', 'recall', 'fscore', 'ROC-AUC'])

print(df_metrics)


end = time.time()
print(df_metrics.iloc[0][0],',',df_metrics.iloc[0][1],',',df_metrics.iloc[0][2],',',df_metrics.iloc[0][3],',',df_metrics.iloc[0][4],',',df_cm.iloc[0][0],',',df_cm.iloc[0][1],',',df_cm.iloc[0][2],',',df_cm.iloc[0][3],',', end-start)


print("Time taken:", end-start)

LR: 1
   True Positives  True Negatives  False Positives  False Negatives
0            9607            9386             1714             3993
   accuracy  precision    recall    fscore   ROC-AUC
0  0.768947     0.8486  0.706397  0.770996  0.861195
0.7689473684210526 , 0.8485999470011483 , 0.7063970588235294 , 0.7709963484611371 , 0.8611950847906731 , 9607 , 9386 , 1714 , 3993 , 0.3417830467224121
Time taken: 0.3417830467224121


Correctly detected attack using query function

In [ ]:


#import modules
import pandas as pd   
import numpy as np
import time

config_file = 'config.txt'
config = pd.read_csv(config_file,sep=',', index_col =None)
attack_id = config.iloc[2,1]

filename = 'data/data_preprocessed_numerical_test_all_features.csv'
    
df_test = pd.read_csv(filename,sep=',')
df_y_pred = pd.read_csv('data/y_pred.csv',sep=',')

df = pd.concat([df_test,df_y_pred], axis = 1)     
df = df.drop(['Unnamed: 0'], axis =1)

df2= df.query('Class_all == @attack_id and Class == 1')

df3= df2.query('Class_all == @attack_id and Class == 1 and y_pred == 1')
      
print("actual # of attack ( attack ", attack_id, "): ", len(df2))
print("correctly detected (attack ", attack_id, "): ", len(df3))

if len(df3)!= 0:
    print("Percentage: ", (len(df3)/len(df2))*100)
else:
    print("Percentage: ", 0)

actual # of attack ( attack  3 ):  1205
correctly detected (attack  3 ):  1066
Percentage:  88.46473029045643
